In [1]:
from typing import cast, Callable
import math
from util.util import Results_Params

def print_table(data: dict[str, list], precision: int = 4):
    if not data:
        print("(empty)")
        return

    # Determine number of rows and ensure index column exists and is first
    n_rows = len(next(iter(data.values())))
    if "index" not in data or len(data["index"]) != n_rows:
        data["index"] = list(range(1, n_rows + 1))
    headers = ["index"] + [h for h in data.keys() if h != "index"]

    # Numeric format
    fmt = f"{{:.{precision}f}}"

    # Detect string columns: if any value is a string, treat entire column as strings
    is_string_col = {
        h: any(isinstance(v, str) for v in data[h])
        for h in headers
    }

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        if is_string_col[h]:
            max_val_len = max(len(str(v)) for v in values)
        else:
            max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row (strings left-aligned, numbers right-aligned)
    header_cells = []
    for h, w in zip(headers, col_widths):
        header_cells.append(f"{h:<{w}}" if is_string_col[h] else f"{h:>{w}}")
    header_row = " | ".join(header_cells)
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row_cells = []
        for h, w in zip(headers, col_widths):
            v = data[h][i]
            if is_string_col[h]:
                row_cells.append(str(v).ljust(w))
            else:
                row_cells.append(fmt.format(v).rjust(w))
        print(" | ".join(row_cells))



In [9]:
from data_handlers.february.coolchic_variance_test import AnalysisHandler as VarianceHandler
from data_handlers.january.coolchic_clic_multiarm import AnalysisHandler as ClicMultiarHandler

def load_helper(path: str, handler_class):
    handler = handler_class(path)
    res_files = handler.load_results()
    raw_results, rate_img_bistream, results_params = handler.parse_results(res_files)
    print(f"Loaded {len(raw_results)} results")
    print("-" * 25)
    return handler.postprocess_results(raw_results), rate_img_bistream, results_params

results_tables = {
    "kodak_variance": load_helper("../../logs_cluster/logs/full_runs/february/2026_02_11_YCoCg_two_by_two_kodak_variance", VarianceHandler),
    "clic_1x1": load_helper("../../logs_cluster/logs/full_runs/january/2026_01_30_YCoCg_one_by_one_Clic", ClicMultiarHandler),
    "clic_2x2": load_helper("../../logs_cluster/logs/full_runs/january/2026_01_15_YCoCg_two_by_two_Clic", ClicMultiarHandler),
    "clic_3x3": load_helper("../../logs_cluster/logs/full_runs/january/2026_01_30_YCoCg_tre_by_tre_Clic", ClicMultiarHandler),
}

Loaded 24 results files.
Loaded 24 results
-------------------------
Loaded 30 results files.
Loaded 30 results
-------------------------
Loaded 30 results files.
Loaded 26 results
-------------------------
Loaded 44 results files.
Loaded 30 results
-------------------------


In [ ]:
def final_prints(results_table: dict[str, list[float | str]], params: Results_Params):
    print(params)
    print_table(results_table)

    for key in results_table.keys():
        avg = sum(cast(list[float], results_table[key])) / len(results_table[key]) if (results_table[key] and isinstance(results_table[key][0], (int, float))) else 0
        std_dev = math.sqrt(sum((x - avg) ** 2 for x in cast(list[float], results_table[key])) / len(results_table[key])) if (results_table[key] and isinstance(results_table[key][0], (int, float))) else 0
        print(f"{key}: {avg:.4f}, {std_dev:.4f}")
    print("-" * 100)

final_prints(results_tables["kodak_variance"][0], results_tables["kodak_variance"][2])
final_prints(results_tables["clic_1x1"][0], results_tables["clic_1x1"][2])
final_prints(results_tables["clic_2x2"][0], results_tables["clic_2x2"][2])
final_prints(results_tables["clic_3x3"][0], results_tables["clic_3x3"][2])

image_index: 5
using_color_space: YCoCg
using_image_arm: True
using_encoder_gain: 64
using_multi_region_image_arm: False
using_multi_region_image_arm_setup: 2x2
using_color_regression: False
total_training_iterations: 142200
total_mac_per_pixel: 1694.3125
  index | Im_name | taskId |   Loss | Rate NN | Rate Latent | Rate Img
--------+---------+--------+--------+---------+-------------+---------
 1.0000 | kodim06 |      9 | 2.9624 |  0.0356 |      0.2315 |   2.6953
 2.0000 | kodim06 |      0 | 2.9627 |  0.0330 |      0.2416 |   2.6880
 3.0000 | kodim06 |     20 | 2.9647 |  0.0345 |      0.2085 |   2.7218
 4.0000 | kodim06 |     14 | 2.9751 |  0.0361 |      0.2327 |   2.7063
 5.0000 | kodim06 |     10 | 2.9854 |  0.0337 |      0.2347 |   2.7170
 6.0000 | kodim06 |      5 | 2.9888 |  0.0297 |      0.2044 |   2.7547
 7.0000 | kodim06 |     13 | 2.9949 |  0.0348 |      0.2268 |   2.7332
 8.0000 | kodim06 |      8 | 2.9966 |  0.0400 |      0.2237 |   2.7329
 9.0000 | kodim06 |     21 | 2.997